# Investigation of citibike ridership with income

# Objective: relate the citibike ridership to income.

## 1-4 are the data wrangling portion of the exercise, and are worth 50% of the points
### In the data wrangling portion points are awarded for data processed so that they support the analysis in parts 5-10. 

## 5-10 are the analysis portion of the exercise, and are worth 50% of the points 
### In the data analysis portion points are awarded for correct analysis, conclusions supported by the analysis *and expressed in full sentences*, and plots that support the analysis and conclusions.


## Data Wrangling

### 1. Download two months of citibike data 201506 and 201601
### 2. Download income information from IRS for NYC and extract the income by zipcode ([alternatives](https://docs.google.com/document/d/1gZ8QwuRYqcwwt2R0vfQFl3RDfEtDpfcii57gJBpWE-M/edit?usp=sharing))
### 3. Find the zipcodes of citibike stations by reverse geocoding the coordinates ([alternatives](https://docs.google.com/document/d/1gZ8QwuRYqcwwt2R0vfQFl3RDfEtDpfcii57gJBpWE-M/edit?usp=sharing))
### 4. Join the datasets and find the number of rides per zipcode in one of your 2015/06 of data (and later in the 2016/01 data)

## Analysis


### 5. Fit a line to ridership (number of rides over one of the 2 months of citibike data downloaded, start with 2015) vs income (total Adjusted gross income for the zip).
### 6. Improve the fit by removing 2 ''suspected'' outliers and *quantify* the improvement
### 7. Fit a 2nd degree polynomial to the same data
### 8. Compare FORMALLY the line and the 2nd degree polynomial fit to assess which is a better fit to your chosen significance level.
### 9. Repeat the analysis with the 2016 citibike dataset. Are the results consistent? What can you say about the outliers?
### 10. Compare the fit to the AGI to a fit to the income per person as endogenous variable  (*i.e.* redo your fit with income per person instead of AGI, see below how to derive income per person) and compare the fits. How do the fit compare? If the latter is better what does this say? If it is worse, what does this say?

## EC: Discuss why it may that the incomePC leads to a different fit and if you have an explaination describe how you would test it, and go ahead and test it.



## 1. Download two months of citibike data ([help](https://docs.google.com/document/d/1GnZ1bU8CilUn7X8ONIOD55fF_fnmGufA0jn8a3__E9w/edit?usp=sharing)) 201506 and 201601. Begin working with 201506, and if you have time repeat the analysis for the other month, to see if your conclusions are robust. 

In [10]:
cb2016.head()

file in place, you can continue


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,923,1/1/2016 00:00:41,1/1/2016 00:16:04,268,Howard St & Centre St,40.719105,-73.999733,3002,South End Ave & Liberty St,40.711512,-74.015756,22285,Subscriber,1958,1
1,379,1/1/2016 00:00:45,1/1/2016 00:07:04,476,E 31 St & 3 Ave,40.743943,-73.979661,498,Broadway & W 32 St,40.748549,-73.988084,17827,Subscriber,1969,1
2,589,1/1/2016 00:00:48,1/1/2016 00:10:37,489,10 Ave & W 28 St,40.750664,-74.001768,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,21997,Subscriber,1982,2
3,889,1/1/2016 00:01:06,1/1/2016 00:15:56,268,Howard St & Centre St,40.719105,-73.999733,3002,South End Ave & Liberty St,40.711512,-74.015756,22794,Subscriber,1961,2
4,1480,1/1/2016 00:01:12,1/1/2016 00:25:52,2006,Central Park S & 6 Ave,40.765909,-73.976342,2006,Central Park S & 6 Ave,40.765909,-73.976342,14562,Subscriber,1952,1


## 2. Downloading income data from IRS 

### Find income data per zipcode in NYC: you can find it from IRS the file name is  14zp33ny 
### and the IRS site is https://www.irs.gov/pub/irs-soi/?C=N;O=D

Use the Adjusted gross income (AGI) for every zipcode. Additionally identify the columns indicating the number of returns, the number of dependents, the number of joint returns. Together they indicate the size of the family unit, allowing you to obtain the income per person in that zipcode, from the AGI of the whole zipcode. 

If you need help look [here]( https://raw.githubusercontent.com/fedhere/PUI2016_solutions/master/midterm/readIncomeXls.py?token=ABnkhnsJkYGifANYhzWZqCBzsOfu_HhPks5YF-VpwA%3D%3D)

*For every zipcode the AGI is the first valid row of the "Adjusted gross income (AGI) [3]" columns associated with that zipcode.* 

Store the income data in a daraframe with (at least) the columns 

**zipcodes,	income, N,	incomePC**

where **zipcodes** are the zipcodes, **income** is the AGI, **N** the number of returns, **incomePC** the AGI for the zipcode divided by (N + N dependents + N joint returns)


<font color="blue">This is a hard dataset to wrangle, but you worked on many difficult datasets already in PUI! But if you think this task is too hard you can download a pre-wrangled version of the dataframe. You will lose 10% of the points. </font>

In [35]:
# After you download it and read it the IRS dataset should look like this
incomeByZip.columns

Index(['Size of adjusted gross income', 'Number of returns',
       'Number of single returns', 'Number of joint returns',
       'Number of head of household returns',
       'Number with paid preparer's signature', 'Number of exemptions',
       'Number of dependents', 'Number of volunteer prepared returns [2]',
       'Unnamed: 10',
       ...
       'Total tax liability [9]', 'Unnamed: 116', 'Additional Medicare tax',
       'Unnamed: 118', 'Net investment income tax', 'Unnamed: 120',
       'Tax due at time of filing [10]', 'Unnamed: 122',
       'Overpayments refunded [11]', 'Unnamed: 124'],
      dtype='object', length=124)

In [6]:
# extract the right entry with iloc[0]: e.g.
# notice in this example the zipcode 10001 is the index of this df row
print ("Adjusted gross income (AGI)  for zipcode 10001:", 
       incomeByZip.loc[[10001]]["Adjusted gross income (AGI) [3]"].iloc[0])

Adjusted gross income (AGI)  for zipcode 10001: 2363960.0


Create a new dataframe zipincome with the value of income per zipcode, and income per person (per capita, PC) per zipcode 
(incomePC = income / (Nreturns + Njoint returns + Ndependents)

In [68]:
# the zip codes series should look like this, and may contain some spurious values (0, 99999)
zipcs[:10]

[0, 99999, 10001, 10002, 10003, 10004, 10005, 10006, 10007, 10009]

In [10]:
# compare your dataframe with mine to know you are on the right track
zipincome.head()

,zipcodes,income,N,Njoint,Ndeps,incomePC
0,0,766646080,9397410,2942890,5539120,42.878688
1,99999,14338084,88940,28130,43810,89.122849
2,10001,2363960,14080,2410,3250,119.754813
3,10002,2215542,43370,11040,19160,30.114748
4,10003,6910992,29810,5460,4790,172.516026


## 3. Find the zipcodes of citibike stations by reverse geocoding the coordinates
You can use the google API including the long and latitide of each station: **CAREFUL!!: you do not need a separate API query per ride, just one per citibike station!** You have a limit of 2500 requests/day, so you cannot submit a request per ride. (You can use pd.DataFrame.drop_duplicates(), for example, to identify identical coordinate pairs or identical station ids, so as to not to repeat queries for the same station)

https://developers.google.com/maps/documentation/geocoding/intro

If you do not have an API key for googlemaps you can get one instantly here
https://developers.google.com/maps/documentation/geocoding/get-api-key


Once you have the zip for a lat/lon pair (lat, lon) you can use a condition like 
```
(cb['start station latitude'] == lat) * (cb['start station longitude'] == lon)
```
as index to identify the rows of the citibike datframe that contain those coordinates and are associated to that zipcode
```
cb['zipcodes'][(cb['start station latitude'] == lat) * (cb['start station longitude'] == lon)] = thatzipcode
```

Once you have the formatted URL you can go [here](https://github.com/fedhere/PUI2016_fb55/blob/master/HW5_fb55/solutions/get_jsonparsed_data.py) for a function to download it. You can use it as revgeo = get_jsonparsed_data(url)["results"][0]['address_components'][-1]['long_name']

You can add a zip column to your dataframe. If you need help go here: https://gist.github.com/fedhere/2d9f835031654eeb0a0d780d203bf224

<font color="blue">You have used APIs before in HW2 and if you did the extra credit in HW5 (and solutions for that HW are posted in the github repo PUI2016_fb55 under HW5_fb55). But if you are not up for using the API you can download the zipcode of each citibike station here 
http://cosmo.nyu.edu/~fb55/UI_CUSP_2015/data/stationsZips2015.json and http://cosmo.nyu.edu/~fb55/UI_CUSP_2015/data/stationsZips2016.json
However, this will cost you 10% of the points.</font>



In [12]:
# you can compare your dataframe with mine to check that you are on track
# notice that some zipcodes may be incorrect: e.g. 2926 is the reverse-geocoded zip I get for station id 128
# later you will have to take care of those
cb2015.head()

//anaconda/lib/python3.5/site-packages/pandas/computation/expressions.py:190: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/pandas/core/internals.py:4417: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  comp = (nn == nn_at)


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,zipcodes
0,1338,6/1/2015 0:00,6/1/2015 0:22,128,MacDougal St & Prince St,40.727103,-74.002971,2021,W 45 St & 8 Ave,40.759291,-73.988597,20721,Subscriber,1984,1,2926
1,290,6/1/2015 0:00,6/1/2015 0:05,438,St Marks Pl & 1 Ave,40.727791,-73.985649,312,Allen St & E Houston St,40.722055,-73.989111,21606,Subscriber,1997,1,10003
2,634,6/1/2015 0:01,6/1/2015 0:11,383,Greenwich Ave & Charles St,40.735238,-74.000271,388,W 26 St & 10 Ave,40.749718,-74.002950,16595,Subscriber,1993,1,10011
3,159,6/1/2015 0:01,6/1/2015 0:04,361,Allen St & Hester St,40.716059,-73.991908,531,Forsyth St & Broome St,40.718939,-73.992663,16949,Subscriber,1981,1,5416
4,1233,6/1/2015 0:02,6/1/2015 0:22,382,University Pl & E 14 St,40.734927,-73.992005,532,S 5 Pl & S 4 St,40.710451,-73.960876,17028,Customer,NaN,0,4510


In [13]:
#grouping and counting (the order may be different if you used the API and if you did not)
# if you need help https://gist.github.com/fedhere/c8dc55be1a2269540df66734a75cff50

cbgroup = cb2015.groupby .... count()
cbgroup.head()

,zipcodes,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,0914,3675,3675,3675,3675,3675,3675,3675,3675,3675,3675,3675,3675,3675,3369,3675
1,10001,51691,51691,51691,51691,51691,51691,51691,51691,51691,51691,51691,51691,51691,45571,51691
2,10002,45970,45970,45970,45970,45970,45970,45970,45970,45970,45970,45970,45970,45970,40795,45970
3,10003,74663,74663,74663,74663,74663,74663,74663,74663,74663,74663,74663,74663,74663,67536,74663
4,10004,13698,13698,13698,13698,13698,13698,13698,13698,13698,13698,13698,13698,13698,9850,13698


### Join the datasets (dataframes)
notice there may be lots of invalid zipcodes from bad reverse geocoding!! Drop all data with zipcodes < 10000. I named my joined DF cbincome

In [20]:
cbincome.head()

,zipcodes,income,N,Njoint,Ndeps,incomePC,Nrides
0,10001,2363960,14080,2410,3250,119.754813,51691
1,10002,2215542,43370,11040,19160,30.114748,45970
2,10003,6910992,29810,5460,4790,172.516026,74663
3,10004,925417,2540,840,1130,205.192239,13698
4,10005,5545849,5890,1340,1340,647.123571,9631


# ANALYSIS PORTION remember to make plots and describe your findings with complete sentences.

## 5. Plot ridership vs AGI and fit the data with a line

You have fitted lines and curves in multiple HWs. If you do it with statsmodels you get a number of diagnostics with the .summary() method, for example

## 6. Choose two high leverage points that may be outliers, fit the data without them and compare the fits (make plots where appropriate)


If you need help on removing data: https://gist.github.com/fedhere/89db40a3c9a28335bf5a35d52a52c595

## 7. Fit a 2nd to the data degree polynomial and assess if the addition of the extra parameter improves the fit  (make plots where appropriate)

## 8. Formally compare the first and second degree polynomial fit and decide which is better and if the addition of the extra model parameter is justified by the data  (make plots where appropriate)

## 9.  Test with 2016, discuss the results  (make plots where appropriate)

What happened to the outliers?

## 10.  Fit the rides to the income per person, discuss the result  (make plots where appropriate)

Income per persoin is incomePC from part 2

## EC: Speculate on reasons why incomePC fit is different from the income fit (if it is). 

## EC: Design and perform a test to explain why the incomePC fit is different from the income fit (if it is)  (make plots where appropriate).